In [ ]:
from dotenv import load_dotenv
import os

model_name=os.environ.get("model_name")
base_url=os.environ.get("base_url")
api_key=os.environ.get("api_key")
print(model_name)
print(base_url)
print(api_key)

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv


load_dotenv() 

BASE_URL = "https://api.siliconflow.cn/v1"
API_KEY = "sk-iujgwjycqgmvzxycfgyynuowipaykmbbcneerzvdnehpqvfs"
EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-0.6B"

# --- 测试函数 ---
def test_custom_embedding():
    """
    使用自定义 base_url 和模型测试 OpenAIEmbeddings。
    """
    test_texts = [
        "你好，世界！",
        "Hello, World!",
        "什么是向量嵌入？"
    ]
    
    print("--- 正在初始化嵌入模型 ---")
    print(f"Base URL: {BASE_URL}")
    print(f"Model: {EMBEDDING_MODEL}")
    
    try:
        # 使用 OpenAIEmbeddings 初始化嵌入模型，并指定自定义参数
        # ⚠️ 注意：api_key 和 openai_api_base 是关键
        embeddings = OpenAIEmbeddings(
            model=EMBEDDING_MODEL,
            openai_api_base=BASE_URL,
            openai_api_key=API_KEY
            # 其他参数（如 chunk_size）如果需要也可以在这里设置
        )
        
        print("\n--- 正在请求嵌入向量 ---")
        
        # 1. 测试单句嵌入
        query = test_texts[0]
        embedding_vector = embeddings.embed_query(query)
        print(f"✅ 成功嵌入查询 '{query}'")
        print(f"向量维度: {len(embedding_vector)}")
        # print(f"部分向量值: {embedding_vector[:5]}...") # 可选：打印部分向量
        
        # 2. 测试多句嵌入 (Documents)
        document_embeddings = embeddings.embed_documents(test_texts)
        print(f"\n✅ 成功嵌入 {len(document_embeddings)} 个文档")
        
        # 检查所有返回的向量是否维度一致
        first_dim = len(document_embeddings[0])
        all_match = all(len(vec) == first_dim for vec in document_embeddings)
        print(f"所有向量维度是否一致: {all_match} (维度: {first_dim})")
        
        print("\n🌟 嵌入模型测试成功！")
        
    except Exception as e:
        # 如果出现 404 Not Found 或其他 API 错误，将在这里捕获
        print(f"\n❌ 嵌入模型测试失败！请检查配置和服务：")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误信息: {e}")
        print("\n💡 检查建议：1. 确认 base_url 和 api_key 正确； 2. 确认模型名称 'Qwen/Qwen3-Embedding-0.6B' 在该服务上可用。")


if __name__ == "__main__":
    test_custom_embedding()

In [ ]:
# 测试RAG服务
from tools.rag_service import RAGService
print(RAGService().get_retriever().invoke("test query"))

In [ ]:
from AgentModule.edu_agent import rag_search
print(rag_search("your question about PDF content"))

In [1]:
import sys
import os

from AgentModule import create_agent
from AgentModule.edu_agent import run_agent
from tools.rag_service import get_rag_service
from frontend_service.interface import run_agent, LanguageHandler 


def main():
    """
    主测试函数
    """
    print("🚀 开始测试对话核心逻辑...")
    agent=create_agent()
    print("✅ Agent 和 Retriever 加载完成。")
    rag_service = get_rag_service()
    retriever = rag_service.get_retriever()

    # --- 3. 定义你要测试的问题列表 ---
    test_questions = [
        "你好",
        "你是谁？",
        # "请介绍一下LangChain。", # 这是一个可能触发检索器（Retriever）的问题
        # "天空为什么是蓝色的？", # 这是一个通识问题
        # "帮我写一首关于秋天的诗。", # 这是一个创造性问题
        # "asdfqwer;lkj", # 这是一个可能触发备用模型（Fallback）的无效问题
    ]
    print(f"\n准备了 {len(test_questions)} 个测试问题。")
    print("-" * 30)

    # --- 4. 循环测试并打印结果 ---
    for i, message in enumerate(test_questions):
        print(f"\n🧪 === 测试案例 {i+1} ===")
        print(f"💬 测试问题: {message}")

        try:
            # 直接调用你的核心函数
            result, used_fallback, used_retriever = run_agent(
                message,
                executor=agent,
                retriever=retriever,
                return_details=True 
            )


            print(f"🤖 模型原始回答: {result}")
            # print(f"🈯️ 中文翻译回答: {final_result}")
            print(f"💡 是否使用了检索器 (Retriever): {used_retriever}")
            print(f"🛡️ 是否使用了备用模型 (Fallback): {used_fallback}")

        except Exception as e:
            print(f"❌ 测试出错: {e}")
            import traceback
            traceback.print_exc()

        print("-" * 30)

    print("\n🎉 所有测试案例执行完毕。")

main()

/opt/anaconda3/envs/LangChain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 开始测试对话核心逻辑...
✅ Agent 和 Retriever 加载完成。

准备了 2 个测试问题。
------------------------------

🧪 === 测试案例 1 ===
💬 测试问题: 你好


/Users/pigpeppa/Downloads/AI-education/tools/rag_utils.py:23: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


Question:     你好

Context:
 -2- 
前实际问题综合能力，最后还要具有把自己的实践过程和结果叙述成文字的写作能力。
通过数学建模全过程的各个环节，学生们进行着创造性的思维活动，模拟了现代科学研
究过程。通过“数学建模”课程的教学和数学建模活动极大地开发了学生的创造性思维
的能力，培养学生在面对错综复杂的实际问题时，具有敏锐的观察力和洞察力，以及丰
富的想象力。因此，“数学建模”课程在培养学生的创新能力方面有着其它课程不可替
代的作用。 
几年的“数学建模”教学实践告诉我们，进行数学建模教学，为学生提供一本内容
丰富，既理论完整又实用的“数学建模”教材，使学生少走弯路尤为重要。这也是我们
编写这本教材的初衷。本教材可以说既是我们多年教学经验的总结，也是我们心血的结
晶。本教材的特点是尽量为学生提供常用的数学方法，并将相应的 Matlab 程序提供给
学生，使学生在进行书中提供的案例的学习中，在自己动手构建数学模型的同时上机进
行数学实验，从而为学生提供数学建模全过程的训练，以便能够达到举一反三，取得事
半功倍的教学效果。司守奎同志编写了全部的 Matlab 程序，参加本书编写的还有毛凯
同志。 
全书共二十四章，各章有一定的独立性，这样便于教师和学生按需要进行选择。完
成本教材的教学大约需要 60 学时，其中方法教学与上机实践的比例一般不应少于
1:1 。 
一本好的教材需要经过多年的教学实践，反复锤炼。由于我们的经验和时间所限，
书中的错误和纰漏在所难免，敬请同行不吝指正。 
 
 
                                                           
                                            编者 
                                         2 0 0 3 年12 月 

 -1- 
 
前    言 
今天，人类社会正处在由工业化社会向信息化社会过渡的变革。以数字化为特征的
信息社会有两个显著特点：计算机技术的迅速发展与广泛应用；数学的应用向一切领域
渗透。随着计算机技术的飞速发展，科学计算的作用越来越引起人们的广泛重视，它已
经与科学理论和科学实验并列成为人们探索和研究自然界、人类社会的三大基本方法

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


Question:     你是谁？


> Entering new AgentExecutor chain...
Thought: 我需要回答“你是谁？”这个问题。这是一个关于我身份的简单问题，不需要使用工具，可以直接作答。
Final Answer: 我是Qwen，是阿里云研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。

> Finished chain.
Output:     我是Qwen，是阿里云研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。
🤖 模型原始回答: 我是Qwen，是阿里云研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。
💡 是否使用了检索器 (Retriever): False
🛡️ 是否使用了备用模型 (Fallback): False
------------------------------

🎉 所有测试案例执行完毕。
